### Урок 2. #Профилирование пользователей. Сегментация аудитории: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)#

#### Пример из жизни (новостной портал)

Представим, что мы - компания-агрегатор новостей (новостной портал).

У нас есть:

1. читатели
2. новости

Для каждого пользователя мы можем за какой-то период (например, 1 день) достать из базы данных список прочитанных им новостей.

Для каждой новости мы можем вытащить текст и метаинформацию.

### Задача #1: нужно построить модель прогнозирования оттока - это наша downstream-задача. 

Нам нужны:

1. векторное представление пользователя
2. сегменты, описывающие интересы пользователя

p.s. в контексте нашей задачи - это одно и то же

### С чего начнем?

С векторного представления и сегментов новостей!

Есть два очевидных варианта как это сделать:

1. многоклассовая классификация
2. кластеризация документов с последующей попыткой их (кластера) интерпретировать

Проблема:

1. для классификации нам нужно сначала разметить новости - привлечение ручного труда

### Задача тематического моделирования

#### Неформально!

- на свете бывают темы (заранее неизвестные), которые отражают то, о чём могут быть части документа;
- каждая тема – это распределение вероятностей на словах, т.е. мешок слов, из которого можно с разной вероятностью вытащить разные слова;
- каждый документ – это смесь тем, т.е. распределение вероятностей на темах, кубик, который можно кинуть;
- процесс порождения каждого слова состоит в том, чтобы сначала выбрать тему по распределению, соответствующему документу, а затем выбрать слово из распределения, соответствующего этой теме.

Вероятностные модели удобно понимать и представлять в виде порождающих процессов (generative processes), когда мы последовательно описываем, как порождается одна единица данных, вводя по ходу дела все вероятностные предположения, которые мы в этой модели делаем. Соответственно, порождающий процесс для LDA должен последовательно описывать, как мы порождаем каждое слово каждого документа. И вот как это происходит (здесь и далее я буду предполагать, что длина каждого документа задана – её тоже можно добавить в модель, но обычно это ничего нового не даёт):

1. для каждой темы t выбираем вектор phi_t - распределение слов в теме

2. для каждого документа d:

    2.1 выбираем Theta_d - распределение тем в документе
    
    2.2 для каждого из слов документа w:
        2.2.1 выбираем тему z~Theta_d
        2.2.2 выбираем слово w~p(w|z, phi)

![lda](lda_simple.png "LDA")

ист. https://habr.com/ru/company/surfingbird/blog/230103/

#### Чуть более формально

Терминология:

1. документ - коллекция слов 
2. тема - набор токенов (слов), совместно часто встречающихся в документах

Более формально:

1. тема - условное распределение на множестве терминов, p(w|t)
2. тематический профиль документа - условное распределение тем p(t|d)

Вопрос: что же нам дано (в терминах условной вероятности)?

Ответ: условное распределение слов в документах - p(w|d)

Прекрасное объяснение от Воронцова - http://www.machinelearning.ru/wiki/images/d/d5/Voron17survey-artm.pdf

Тематическая модель позволяет нам получить p(w|t), p(t|d) по известным p(w|d)

![ab_split](tm1.png "TM1")

### Переходим к практике

In [1]:
import pandas as pd

Наши новости

In [2]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [3]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
tfidf = TfidfVectorizer()

In [6]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [7]:
#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [8]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [9]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [10]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [11]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-10-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


CPU times: user 24.4 s, sys: 147 ms, total: 24.5 s
Wall time: 24.5 s


In [12]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 3min 8s, sys: 156 ms, total: 3min 8s
Wall time: 3min 8s


А теперь в 3 строчки обучим нашу модель

In [13]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [14]:
common_dictionary[11]

'взаимодействие'

Все просто - это словарь наших слов

Запускаем обучение

In [15]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

CPU times: user 1min 21s, sys: 1.59 s, total: 1min 22s
Wall time: 32.5 s


In [16]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [17]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(3, 0.43849948),
 (7, 0.23992786),
 (10, 0.06476773),
 (12, 0.13363081),
 (15, 0.06992505),
 (16, 0.033012938)]

In [18]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: рейс физика билет конструкция турецкий соцсеть берлин
topic_1: исследование россия nn территория население данные статья
topic_2: украина банк российский экономика наука гражданин украинский
topic_3: первый nn товар автор всё большой день
topic_4: участок обращение стоимость китайский превысить океан азия
topic_5: квартира испытание памятник городской двигатель век свет
topic_6: убийство сотрудник остров обвинение следователь подозревать арестовать
topic_7: рубль россия млрд проект новый фонд тыс
topic_8: закон пенсия nn решение власть право орган
topic_9: nn станция путин белоруссия улица открытие белорусский
topic_10: доллар источник метод первый болезнь технология новый
topic_11: миссия дональд расчёт снижение новый сенатор озеро
topic_12: всё научный ребёнок очень большой мозг случай
topic_13: северный южный рейтинг бомба место германия nn
topic_14: журнал космос министерство дыра земля источник поиск
topic_15: погибнуть взрыв журнал вода эксперимент катастрофа расстояние


Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [19]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [20]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.212025,0.675292,0.000000,0.0,0.0,0.0,0.000000,0.00000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.045905
1,4896,0.0,0.000000,0.000000,0.109399,0.0,0.0,0.0,0.000000,0.09424,...,0.545520,0.000000,0.0,0.000000,0.0,0.106192,0.0,0.0,0.000000,0.000000
2,4897,0.0,0.000000,0.000000,0.438611,0.0,0.0,0.0,0.239967,0.00000,...,0.069862,0.033021,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,4898,0.0,0.000000,0.000000,0.459455,0.0,0.0,0.0,0.000000,0.00000,...,0.000000,0.000000,0.0,0.387395,0.0,0.000000,0.0,0.0,0.026393,0.000000
4,4899,0.0,0.000000,0.971707,0.000000,0.0,0.0,0.0,0.000000,0.00000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [21]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [22]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [23]:
doc_dict[293622]

array([0.        , 0.10495654, 0.        , 0.37299007, 0.        ,
       0.        , 0.        , 0.07004792, 0.07368814, 0.06478011,
       0.        , 0.        , 0.        , 0.06559564, 0.02785048,
       0.        , 0.        , 0.        , 0.02663243, 0.        ,
       0.        , 0.01941594, 0.10781654, 0.        , 0.05670549])

In [24]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list, func=np.mean):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = func(user_vector, 0)
    return user_vector

In [25]:
get_user_embedding(user_articles_list)

array([0.        , 0.15345336, 0.12553515, 0.00983904, 0.        ,
       0.        , 0.03463202, 0.08077216, 0.12543   , 0.05544861,
       0.01407637, 0.00428953, 0.05301258, 0.02543478, 0.00233886,
       0.00350249, 0.02651017, 0.        , 0.17250672, 0.        ,
       0.        , 0.0152263 , 0.05276331, 0.03220047, 0.        ])

Интересовался новостями с топиками topic_3, topic_14 (что-то про политику и государство)

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [26]:
def process_users(func, df=users):
    user_embeddings = pd.DataFrame([i for i in df['articles'].apply(lambda x: get_user_embedding(x, func), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = df['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
    return user_embeddings

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [28]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [64]:
target = pd.read_csv("users_churn.csv")

def eval_model(func, method):
    if method=='normal':
        user_embeddings = process_users(func)
    elif method=='tfidf':
        user_embeddings = process_tfidf(users)
    X = pd.merge(user_embeddings, target, 'left')
    X_train, X_test, y_train, y_test = train_test_split(X.drop(columns=['churn', 'uid']), 
                                                    X['churn'], random_state=0)
    logreg = LogisticRegression()
    #обучим 
    logreg.fit(X_train, y_train)
    preds = logreg.predict_proba(X_test)[:, 1]
    
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    pred_labels = [int(i>=thresholds[ix]) for i in preds]
    roc_auc = roc_auc_score(y_test, pred_labels)
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, ROC_AUC=%.3f' % (thresholds[ix], 
                                                                                          fscore[ix],
                                                                                          precision[ix],
                                                                                          recall[ix],
                                                                                          roc_auc))
    return {'func': func.__name__,
            'ROC_AUC':roc_auc, 
            'Precision': precision[ix], 
            'Recall': recall[ix], 
            'F_score': fscore[ix]}

In [65]:
task_1 = pd.DataFrame(columns=['func','F_score', 'Precision', 'Recall', 'ROC_AUC'])
for i in [np.mean, np.max, np.median]:
    task_1 = task_1.append(eval_model(i, 'normal'), ignore_index=True)

task_1

Best Threshold=0.233666, F-Score=0.633, Precision=0.564, Recall=0.722, ROC_AUC=0.822
Best Threshold=0.324547, F-Score=0.776, Precision=0.754, Recall=0.800, ROC_AUC=0.882
Best Threshold=0.269126, F-Score=0.760, Precision=0.738, Recall=0.784, ROC_AUC=0.872


,func,F_score,Precision,Recall,ROC_AUC
0,mean,0.633274,0.563694,0.722449,0.822193
1,amax,0.776238,0.753846,0.800000,0.881766
2,median,0.760396,0.738462,0.783673,0.872464


### Видно, что все метрики значительно выросли, лучшей функцией является то медиана, то np.max, при разных запусках ноутбука

In [66]:
def process_tfidf(df=users):
    df_copy = df.copy()
    df_copy['articles'] = df_copy['articles'].apply(lambda x: x.replace('[', '').replace(']', ''))
    df_copy = df_copy.join(pd.DataFrame(tfidf.fit_transform(df_copy['articles']).toarray(), 
                                        columns=tfidf.get_feature_names())).drop(columns='articles')
    return df_copy

In [68]:
task_2 = pd.DataFrame(columns=['func','F_score', 'Precision', 'Recall', 'ROC_AUC'])
for i in [np.mean, np.max, np.median]:
    task_2 = task_2.append(eval_model(i, 'tfidf'), ignore_index=True)

task_2

Best Threshold=0.152747, F-Score=0.932, Precision=0.945, Recall=0.918, ROC_AUC=0.955
Best Threshold=0.152747, F-Score=0.932, Precision=0.945, Recall=0.918, ROC_AUC=0.955
Best Threshold=0.152747, F-Score=0.932, Precision=0.945, Recall=0.918, ROC_AUC=0.955


,func,F_score,Precision,Recall,ROC_AUC
0,mean,0.931677,0.945378,0.918367,0.95548
1,amax,0.931677,0.945378,0.918367,0.95548
2,median,0.931677,0.945378,0.918367,0.95548


### Классификатор показывает гораздо более высокие метрики

#### Библиотеки, которые нужно установить:

1. gensim
2. razdel
3. pymorphy2
4. nltk